In [35]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import math

In [36]:
options = Options() 

options.add_experimental_option("prefs", {})
# options.add_argument("--headless") ## Abrirlo en modo fantasma
driver = webdriver.Chrome(options=options, executable_path=r"C:\chromeDriver\chromedriver.exe")

In [37]:
driver.get("https://ec.europa.eu/eures/eures-searchengine/page/main#/search")

## Selección de filtros

In [38]:
def selFiltro(filtro, text):
    if text == "":
        expand = filtro.find_element_by_class_name("expand")
        expand_button = expand.find_element_by_class_name("expandFacet")
        expand_button.click()
    els = filtro.find_elements_by_class_name("hover-color")
    for el in els:
        if text != "" and el.text.find(text) >= 0:
            driver.execute_script("arguments[0].click();", el);
            break
        elif text == "":
            ## Si viene blanco selecciono todos los visibles
            driver.execute_script("arguments[0].click();", el);
    waitLoader()

## Función de espera del loader

In [39]:
def waitLoader():
    ## Espero a que el loader desaparezca
    time.sleep(1) ## Le meto 1 segundo de espera para que de tiempo a que el loader aparezca
    WebDriverWait(driver, 10).until(EC.invisibility_of_element((By.ID, "loader")))

## Clase para guardar los jobs

In [40]:
class Job(object):
    def __init__(self, titulo, lugaryfecha, categorias):
        self.titulo = titulo
        self.lugaryfecha = lugaryfecha
        self.categorias = categorias

In [41]:
try:
    element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "facet-POSITION_LOCATION")))
finally:
    time.sleep(5)
    filtros = driver.find_elements_by_class_name("m1-level-0")
    
    ## Selecciono el país
    selFiltro(filtros[0], "Germany")
    ## Selecciono el filtro EURES flag
    selFiltro(filtros[1], "Without")
    ## Selecciono el filtro tipo de contrato
    selFiltro(filtros[2], "Full-time")
    ## Selecciono el filtro de nivel de educación
    selFiltro(filtros[3], "Upper secondary")
    ## Selecciono el filtro de posicion
    selFiltro(filtros[4], "Contract")
    ## Selecciono el filtro de experiencia
    selFiltro(filtros[5], "Not")

    ## Saco el número de trabajos que existen para esos filtros
    jobs_resume = driver.find_element_by_class_name("small-content")
    vacances = jobs_resume.find_element_by_class_name("ng-binding")
    n_vacances = vacances.text.split(" ")[0];
    n_vacances = n_vacances.replace(",", "")
    print("Se han encontrado " + n_vacances + " ofertas para los filtros seleccionados.")
    
    n_pages = int(n_vacances) / 10
    print("Hay " + str(math.ceil(n_pages)) + " paginas")

Se han encontrado 115 ofertas para los filtros seleccionados.
Hay 12 paginas


## Pasar de página y obtener los datos de cada una de las ofertas existentes

In [42]:
    cont_pages = 1
    c = pd.DataFrame(columns=['titulo', 'lugaryfecha', 'categorias'])
    cols = list(c)
    jobsArr = []
    while cont_pages < math.ceil(n_pages):
        cont_pages = cont_pages + 1
        jobs_resume = driver.find_element_by_class_name("small-content")
        jobs = driver.find_elements_by_class_name("job-container")
        for job in jobs:                
            title = job.find_element_by_tag_name("jv-title");
            place_date = job.find_elements_by_tag_name("div")[1]
            categories = job.find_element_by_id("jobCategoriesCodes")
            jobsArr.append(Job(title.text, place_date.text, categories.text))
        next_page = jobs_resume.find_element_by_class_name("next")
        time.sleep(5)
        next_page.click()
        waitLoader()

    dataframe = pd.DataFrame([[getattr(i,j) for j in cols] for i in jobsArr], columns = cols)

In [43]:
dataframe.head()

,titulo,lugaryfecha,categorias
0,MONTER/MONTERKA INDUSTRIJSKIH STROJEVA,Germany (DE) 1 day(s) ago (17/10/2019),Category: Toolmakers and related workers / Agr...
1,TIG ZAVARIVAČ/ZAVARIVAČICA,Germany (DE) 1 day(s) ago (17/10/2019),Category: Welders and flamecutters
2,ZAVARIVAČ/ICA,Germany (DE) 1 day(s) ago (17/10/2019),Category: Welders and flamecutters
3,KONOBAR/KONOBARICA,Germany (DE) 1 day(s) ago (17/10/2019),Category: Waiters
4,ZAVARIVAČ/ICA - BRUSAČ/ICA,Germany (DE) 1 day(s) ago (17/10/2019),Category: Welders and flamecutters / Metal wor...


In [44]:
dataframe.to_csv("eures.csv", sep=';')